In [146]:
import mysql.connector
import pandas as pd
import numpy as np

In [147]:
sakila_db = mysql.connector.connect(
    host="127.0.0.1", 
    user="root", 
    password="Bismillah@99",
    database='sakila',
    auth_plugin="mysql_native_password"
)

In [148]:
world_db = mysql.connector.connect(
    host="127.0.0.1", 
    user="root", 
    password="Bismillah@99",
    database='world',
    auth_plugin="mysql_native_password"
)

In [149]:
cursor_sakila = sakila_db.cursor()
def sql_sakila(write_query):
    cursor_sakila.execute(write_query)
    result = cursor_sakila.fetchall()
    df = pd.DataFrame(result, columns = cursor_sakila.column_names)
    return df

In [150]:
cursor_world = world_db.cursor()
def sql_world(write_query):
    cursor_world.execute(write_query)
    result = cursor_world.fetchall()
    df = pd.DataFrame(result, columns = cursor_world.column_names)
    return df

In [151]:
# 6
# Tampilkan 15 daftar film yang memiliki huruf "K" pada akhir title, 
# lalu tampilkan title, durasi, serta Bahasa pada film
data = sql_sakila("""
                SELECT title AS film_title, length AS duration, l.name AS language_name
                FROM sakila.film f, language l
                WHERE title LIKE '%K'
                AND f.language_id = l.language_id
                LIMIT 5;
            """)
data

,film_title,duration,language_name
0,AIRPORT POLLOCK,54,English
1,BEAST HUNCHBACK,89,English
2,BULL SHAWSHANK,125,English
3,CANYON STOCK,85,English
4,CHILL LUCK,142,English


In [152]:
#2

data = sql_world("""
                SELECT country.name AS nama_negara,
                    country.continent AS nama_benua,
                    country.lifeexpectancy,
                    country.gnp
                FROM country
                WHERE country.continent = 'Asia'
                AND country.lifeexpectancy > (SELECT AVG(lifeexpectancy)
                                            FROM country
                                            WHERE continent = 'Europe')
                LIMIT 10;
            """)
data

,nama_negara,nama_benua,lifeexpectancy,gnp
0,Cyprus,Asia,76.7,9333.00
1,Hong Kong,Asia,79.5,166448.00
2,Israel,Asia,78.6,97477.00
3,Jordan,Asia,77.4,7526.00
4,Japan,Asia,80.7,3787042.00
5,Kuwait,Asia,76.1,27037.00
6,Macao,Asia,81.6,5749.00
7,Singapore,Asia,80.1,86503.00
8,Taiwan,Asia,76.4,256254.00


In [153]:
data = sql_sakila('''
                    WITH AvgRental AS (
                        SELECT AVG(rental_duration) AS avg_duration
                        FROM film
                    )
                    SELECT
                        RANK() OVER (ORDER BY SUM(rental_duration) DESC) AS rank1,
                        category.name AS film_name,
                        SUM(rental_duration) AS total_rental_duration,
                        SUM(SUM(rental_duration)) OVER (ORDER BY SUM(rental_duration) DESC) AS cumulative_sum,
                        AVG(SUM(rental_duration)) OVER (ORDER BY SUM(rental_duration) DESC) AS moving_avg
                    FROM film
                    JOIN film_category ON film.film_id = film_category.film_id
                    JOIN category ON film_category.category_id = category.category_id
                    WHERE rental_duration <= (SELECT avg_duration FROM AvgRental)
                    GROUP BY category.name
                    ORDER BY total_rental_duration DESC
                    LIMIT 5;
                ''')
data

,rank1,film_name,total_rental_duration,cumulative_sum,moving_avg
0,1,Documentary,114,114,114.0000
1,2,Sports,110,224,112.0000
2,3,New,107,331,110.3333
3,4,Animation,102,433,108.2500
4,5,Foreign,95,528,105.6000
